# Settings

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os

# Task 1

$A \cdot x = b$

$A = \left(\begin{matrix}1 & 1 \\ 0 & 0.1 \end{matrix}\right); b = \left(\begin{matrix}[4.1, 3.9] \\ [0.1, 0.2]\end{matrix}\right)$

## Create A & b

In [2]:
a_slae = np.array([[1, 1], [0, 0.1]])

inf_b = np.array([4.1, 0.1])
sup_b = np.array([3.9, 0.2])

## Define sti

$KR^n \rightarrow R^{n\times n}$

In [3]:
def sti_vector(x_inf, x_sup):
    return np.hstack([-x_inf, x_sup])

def back_sti_vector(sti_vec):
    half = sti_vec.shape[0] // 2
    x_inf = -sti_vec[:half]
    x_sup = sti_vec[half:]
    return x_inf, x_sup

def sti_dot_mtx(a_mtx):
    a_plus = np.zeros(a_mtx.shape)
    a_minus = np.zeros(a_mtx.shape)
    for i in range(a_mtx.shape[0]):
        for j in range(a_mtx.shape[1]):
            if (a_mtx[i][j] > 0):
                a_plus[i][j] = a_mtx[i][j]
            else:
                a_minus[i][j] = -a_mtx[i][j]
    return np.block([[a_plus, a_minus], [a_minus, a_plus]])

### Test sti

In [4]:
tmp = np.array([1, -3])
tmp_sup = np.array([2, 4])
sti = sti_vector(tmp, tmp_sup)
print(sti)
tmp, tmp_sup = back_sti_vector(sti)
print(tmp)
print(tmp_sup)

a_mtx = np.array([[1, 2], [-3, 4]])
print(sti_dot_mtx(a_mtx))

[-1  3  2  4]
[ 1 -3]
[2 4]
[[1. 2. 0. 0.]
 [0. 4. 3. 0.]
 [0. 0. 1. 2.]
 [3. 0. 0. 4.]]


## Calculate $G(y)$, $y \in R^{n \times n}$

In [5]:
def compute_G(C_mtx, cur_point, d_sti):
    intervaled_inf, intervaled_sup = back_sti_vector(cur_point)
    return sti_vector(np.dot(C_mtx, intervaled_inf), np.dot(C_mtx, intervaled_sup)) - d_sti 

## Calculate $x^{(0)} \in R^{n \times n}$

In [6]:
def compute_x0(mid_C, sti_d):
    return np.linalg.solve(mid_C, sti_d)

## Calculate subgradient D

In [7]:
def compute_D(D, i, j, a_ij, b_inf, b_sup):
    n = D.shape[0] // 2;
    
    if a_ij > 0:
        D[i, j] = a_ij
        D[i + n, j + n] = a_ij
    else:
        D[i, j + n] = a_ij
        D[i + n, j] = a_ij
    return D

# Subgradient Newton method

In [8]:
def subdiff2(A, inf_b, sup_b):
    # params
    learning_rate = 1
    accuracy = 1e-10
    max_iter = 50
    
    n = A.shape[0]  # dim
    
    # compute x0
    C_sti = sti_dot_mtx(A)  # for x0
    d_sti = sti_vector(inf_b, sup_b)  # for x0
    cur_x = compute_x0(C_sti, d_sti)  # x0: (mid C)~ x0 = sti(b) 
    #cur_x = np.zeros(d_sti.shape[0])  # x0 = 0
    
    prev_x = cur_x
    started = False  # to enter the first iteration

    # do until method converges
    cur_iter = 0
    while not started or np.linalg.norm(cur_x - prev_x) > accuracy:
        started = True
        cur_iter += 1
        if (cur_iter > max_iter):
            print("Too many iterations")
            break
        prev_x = cur_x  # update after the previous step
        D = np.zeros((2 * n, 2 * n))  # subgrad mtx
        for i in range(n):
            for j in range(n):
                g = A[i, j]
                h_inf = -prev_x[j]
                h_sup = prev_x[j + n]
                D = compute_D(D, i, j, g, h_inf, h_sup)
        G_part = compute_G(A, prev_x, d_sti)
        dx = np.linalg.solve(D, -G_part)
        cur_x = prev_x + learning_rate * dx
    print(f"Iteration count: {cur_iter}")
    return back_sti_vector(cur_x)

# Test subgrad Newton method

In [9]:
x_inf, x_sup = subdiff2(a_slae, inf_b, sup_b)
print(f"X_inf: {x_inf}")
print(f"X_sup: {x_sup}")

Iteration count: 1
X_inf: [3.1 1. ]
X_sup: [1.9 2. ]


# Task 2

## Get square matrices

In [10]:
def get_square(mtx):
    mtx_shape = mtx.shape
    result_n = min(mtx_shape[0], mtx_shape[1])
    return mtx[:result_n,:result_n]

## Make diagonal dominant matrices
### Method requires diagonal dominant matrices

In [11]:
# check if matrix diagonal dominant
def diagonal_dominant(mtx):
    D = np.diag(np.abs(mtx)) # get diagonal
    E = np.sum(np.abs(mtx), axis=1) - D # row sum without diagonal elements
    return np.all(D > E)  # true if the matrix is diagonal dominant

# make matrix diagonal dominant
# (add diagonal matrix to achieve diagonal domination)
def make_diagonal_dominant(mtx):
    if diagonal_dominant(mtx):
        return mtx  # don't need to change matrix
    D = np.diag(np.abs(mtx))  # get diagonal
    E = np.sum(np.abs(mtx), axis=1) - D  # row sum
    eps = 0.1  # diag - sum > eps
    for i in range(mtx.shape[0]):
        if D[i] - E[i] <= eps:  # find bad lines
            mtx[i,i] += E[i] - D[i] + eps  # increase diag elements
    return mtx

## Generate right part

In [12]:
def generate_right_part(mtx):
    n = mtx.shape[0]
    x = np.random.uniform(low=1, high=5, size=n)
    b = np.dot(mtx, x)  # get right part
    rads = np.random.uniform(low=0.5, high=2, size=n)  # generate radiuses for b
    b_inf = b - rads  # make b interval
    b_sup = b + rads
    return b_inf, b_sup

## Create & solve task

In [13]:
def solve_task_2(mtx):
    mtx = make_diagonal_dominant(mtx)
    b_inf, b_sup = generate_right_part(mtx)
    x_inf, x_sup = subdiff2(mtx, b_inf, b_sup)
    print(f"X inf: {x_inf}")
    print(f"X sup: {x_sup}")
    return x_inf, x_sup

In [14]:
def completeSolveBoth(outdir):
    # Read mtx from file
    in_dir = 'input'
    mtx_file_1 = 'matrix_n_phi_1.txt'
    mtx_file_2 = 'matrix_n_phi_6.txt'

    nphi_1 = np.loadtxt(os.path.join(in_dir, mtx_file_1))
    nphi_6 = np.loadtxt(os.path.join(in_dir, mtx_file_2))
    
    # see shapes
    print(f"Matrix 1 shape: {nphi_1.shape}")
    print(f"Matrix 2 shape: {nphi_6.shape}")
    
    # get square matrixes
    a_slae_2 = get_square(nphi_1)
    a_slae_3 = get_square(nphi_6)
    
    print("System 1 solution")
    x_inf, x_sup = solve_task_2(a_slae_2)
    # save
    np.savetxt(os.path.join(outdir, 'x_inf_n_phi_1.txt'), x_inf)
    np.savetxt(os.path.join(outdir, 'x_sup_n_phi_1.txt'), x_sup)
    print("System 2 solution")
    x_inf, x_sup = solve_task_2(a_slae_3)
    np.savetxt(os.path.join(outdir, 'x_inf_n_phi_6.txt'), x_inf)
    np.savetxt(os.path.join(outdir, 'x_sup_n_phi_6.txt'), x_sup)

In [15]:
completeSolveBoth('output')

Matrix 1 shape: (256, 36)
Matrix 2 shape: (256, 216)
System 1 solution
Iteration count: 1
X inf: [4.08766262 1.50316998 3.05768254 2.8546778  3.05390663 3.30747949
 3.39752593 1.45962219 2.73264037 1.97179992 2.10249039 1.27773624
 0.99635419 1.77562454 3.68109665 3.60175836 1.11087545 3.12191157
 2.72160374 2.80440369 2.72823141 1.79114188 2.39584973 3.34976705
 2.16703676 2.01464511 3.68582442 3.60253633 3.4045992  2.78468164
 3.58013559 1.60733654 0.5391543  2.8122093  3.5170281  1.37547375]
X sup: [4.96149222 1.73252108 4.74142578 3.06435939 4.38603781 3.05116618
 4.27376602 2.36952541 2.69628867 2.80323139 3.3293706  2.42752173
 1.87608367 2.10758088 3.34173689 3.9458301  2.38722275 4.78625063
 3.73355517 3.37043218 3.81334377 2.85877826 2.93469157 3.28378598
 1.82648991 2.74396665 4.79329398 4.08702085 3.72011221 3.01121542
 3.70861198 2.46121458 3.15265621 4.07688784 4.11222484 2.57796479]
System 2 solution
Iteration count: 1
X inf: [4.02447496 0.87062884 3.2579628  2.19321517 4